### Chapter 8: Writing Fast code

The rest of this chapter talks about important aspects of writing fast code.  We will walk through how to sum the first $n$ whole numbers in the fastest way.  In each case, we will make a function (called `sum1, sum2, ...`) and then test the overall speed of the function and discuss why features are faster than others. We will use the macro `@time` which will determine the time it takes to run. 

#### Sum Function 1
Consider first a for loop:

In [ ]:
function sum1(n::Int)
  local arr = collect(1:n)
  local sum = 0
  for i in arr
    sum += i
  end
  sum
end

This function takes in a positive number `n`, creates an array and then sums the elements of the array as a for loop.  We do some tests with this using the `@time` macro:

In [ ]:
@time sum1(1_000_000)

In [ ]:
@time sum1(10_000_000)

In [ ]:
@time sum1(100_000_000)

In [ ]:
@time sum1(1_000_000_000)

Notice though that in parentheses, it says the number of allocations.  Since the first one had an array of 1 million 64-bit integers (or 8 bytes), it is almost 8 megabytes. 

The factor of 100 higher created an array of 762 megabytes, which is not insignificant.  In short, it is expensive to allocation memory.

In [ ]:
function sum2(n::Integer)
    local sum = 0
    for i=1:n
        sum+=i
    end
    sum
end

This function doesn't use an array, since we don't really need one.  Let's see what happens

In [ ]:
@time sum2(100_000_000)

In [ ]:
@time sum2(1_000_000_000)

In [ ]:
@time sum2(10_000_000_000)

Why is this one faster than `sum1` ? What's going on with the last one?

In [ ]:
function sum3(n::Int)
    local sum = big(0)
    for i=1:n
        sum+=i
    end
    sum
end

If you said "overflow" in the above section, you win a prize--although I don't have a prize to give. :( 

Generally, if overflow is a problem, let's switch to `BigInt`s like above

In [ ]:
@time sum3(1_000_000)

In [ ]:
@time sum3(100_000_000)

We aren't going to have overflow problems, but you should notice that it is much slower to do operations with `BigInt`s. 

#### Exercise
Write a function similar to `sum3` however use `Int128` as the result (this should be the zero for local sum variable.) Call this `sum4` and time it comparsed to both `sum2` and `sum3`. 

Let's try using the `reduce` function:

In [ ]:
function sum5(n::Int)
  reduce(+,1:big(n))
end

In [ ]:
@time sum5(1_000_000)

In [ ]:
@time sum5(10_000_000)

Note, this is much slower than the `sum3` method.  Let's try the built-in `sum` function:

In [ ]:
@time sum(1:big(10)^6)

In [ ]:
@time sum(1:big(10)^20)

In [ ]:
@time sum(1:big(10)^40)

In [ ]:
@time sum(1:big(10)^100)

What's going on?

### Summary of fast code:

* Stick with `Int64` if possible.  Always faster than `BigInt`
* don't create an array unless you need to.  Allocating memory is a slow process.
* use the built-in methods whenever possible.  They have been optimized. Julia is often super smart about some operations.

### 7.7: Computing Fibonacci Numbers

Let's look at the fibonacci numbers.  If $f_1=1,f_2=1$, then 
$$f_n=f_{n-1}+f_{n-2}\qquad\text{for $n\geq2$}$$

Let's find a fibonacci function that is recursive:

In [ ]:
function fibonacci(n::Integer)
    if n <= 2
        return 1
    else
        return fibonacci(n-1) + fibonacci(n-2)
    end
end

The first 10 can be found in the following way:

In [ ]:
map(fibonacci,1:10)

An alternative is:

In [ ]:
fibonacci(n::Int) = n<=2 ? 1 : fibonacci(n-1) + fibonacci(n-2)

This seems reasonable, but if we find the 40th one:

In [ ]:
@time fibonacci(40)

In [ ]:
@time fibonacci(41)

In [ ]:
@time fibonacci(42)

This isn't looking good. I'm sure (without trying we can find the 100th one.) Why is this so slow?

We going to see how many function evaluations are made.  Consider the adapted fibonacci code to compute the number of times it is evaluated:

In [ ]:
function fibonacciEval(n::Integer)
  global num_evals
  if n==1 || n==2
    num_evals +=1
    return 1
  else
    num_evals += 2
    return fibonacciEval(n-1) + fibonacciEval(n-2)
  end
end

In [ ]:
num_evals=0
fibonacciEval(5)
num_evals

In [ ]:
num_evals=0
fibonacciEval(20)
num_evals

In [ ]:
num_evals=0
fibonacciEval(21)
num_evals

Consider the following fibonacci based on a for loop:

In [ ]:
function fibonacci2(n)
  local x,y = (1,1)
  for i = 1:n-1
    x,y = (y, x+y)
  end
  x
end

In [ ]:
@time fibonacci2(50)

In [ ]:
@time fibonacci2(100)

#### Summary of Recursive functions
- Often writing recursive functions is easy, especially if that is the way they are defined. 
- However a short recursive function is not necessarily fast. 